In [1]:
import subprocess
from pathlib import Path
from multiprocessing import Pool

In [26]:
import pandas as pd

In [2]:
def run_mash(query, reference, output):
    with open(output, 'w') as handle:
        subprocess.run(['mash', 'dist', reference, query], stdout=handle)

In [94]:
dirpath = Path('/media/Central_Lab_Storage/NcbiSRA/NCBI_Salmonella_enterica_Albany_SRA/Contigs')
outpath = Path('/media/Central_Lab_Storage/NcbiSRA/NCBI_Salmonella_enterica_Albany_SRA/Mash')
mash_sketch = '/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq.msh'

In [95]:
with Pool(32) as p:
    try:
        for query in dirpath.iterdir():
            output = outpath/(query.stem + '.txt')
            p.apply_async(run_mash, (query, mash_sketch, output))
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [96]:
dirpath = Path('/media/Central_Lab_Storage/NcbiSRA/NCBI_Salmonella_enterica_Albany_SRA/Mash')

data = []
for i in dirpath.iterdir():
    if i.stat().st_size is 0:
        continue
    with open(i) as f:
        min_mash_dist = min(float(line.split()[2]) for line in f)
    data.append((i.stem, min_mash_dist))

In [97]:
df = pd.DataFrame(data, columns=['query', 'mash_dist'])

In [104]:
df[df['mash_dist']>0.01]

,query,mash_dist
18,SRR7187128,0.012025
37,SRR7533272,0.011428
41,SRR7828394,0.010810
154,SRR1958365,0.011446
203,SRR3120729,0.018325


In [88]:
df.to_csv('/media/Central_Lab_Storage/NcbiSRA/NCBI_Neisseria_meningitidis_SRA/Mash.txt', sep='\t', index=False)

In [105]:
mash_sketch = '/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq.msh'

In [106]:
output = subprocess.check_output(['mash', 'dist', '-p', '32', mash_sketch, mash_sketch])

In [112]:
df = pd.DataFrame([line.split() for line in output.decode().splitlines()], columns=['query', 'reference', 'mash-dist', 'p-value', 'matching-hashes'])

In [115]:
df = df[df['query']!=df['reference']]

In [119]:
df['mash-dist'] = df['mash-dist'].astype(float)

/home/chen1i6c04/miniconda3/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [126]:
df.groupby('query')['mash-dist'].min().head()

query
/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq/GCF_000018625.1.fa    0.005421
/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq/GCF_000018705.1.fa    0.008216
/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq/GCF_000020705.1.fa    0.006430
/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq/GCF_000240905.2.fa    0.009190
/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/RefSeq/GCF_000430125.1.fa    0.012672
Name: mash-dist, dtype: float64